##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

注：この例は、Jupyter スタイルのノートブックで今すぐ実行できます。セットアップは必要ありません。「Google Colabで実行」をクリックするだけです

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/model_analysis/tfma_basic"> <img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tfx/tutorials/model_analysis/tfma_basic.ipynb"> <img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tfx/tutorials/model_analysis/tfma_basic.ipynb"> <img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
<td><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tfx/tutorials/model_analysis/tfma_basic.ipynb"><img width="32px" src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table></div>

# TensorFlow Model Analysis

***TensorFlow Extended (TFX) の主要コンポーネントの例***

[TensorFlow Model Analysis (TFMA)](https://www.tensorflow.org/tfx/guide/tfma)は、データのさまざまなスライスにわたってモデル評価を実行するためのライブラリです。TFMA は、[Apache Beam](https://beam.apache.org/documentation/programming-guide/)を使用して、大量のデータを分散して計算を実行します。

この例のcolabノートブックでは、TFMAを使用して、データセットの特性に関してモデルのパフォーマンスを調査および視覚化する方法を示します。以前にトレーニングしたモデルの結果を利用します。シカゴ市からリリースされた[Taxi Trips データセット](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew)を使用する[ Chicago Taxi Example ](https://github.com/tensorflow/tfx/tree/master/tfx/examples/chicago_taxi_pipeline)のためにトレーニングしたモデルを使います。全データセットについては [ BigQuery UI](https://bigquery.cloud.google.com/dataset/bigquery-public-data:chicago_taxi_trips) を参照してください。

モデラーおよび開発者の皆さんは、このデータがどのように使用されるか、モデルの予測が引き起こす可能性のある潜在的メリット・デメリットについて考えてください。このようなモデルは、社会的バイアスと格差を強化する可能性があります。特徴は解決しようとする問題に関連していますか、それともバイアスを導入しますか？ 詳細については、<a target="_blank" href="https://developers.google.com/machine-learning/fairness-overview/">機械学習における公平性</a>についてご一読ください。

注意：TFMAとそれがApache Beamでどのように機能するかを理解するには、ApacheBeam についての知識が少し必要です。ApacheBeam の基本的なコンセプトについては <a target="_blank" href="https://beam.apache.org/documentation/programming-guide/">Beam プログラミングガイド</a>を参照してください。

データセットの列は次のとおりです。

<table>
<tr>
<td>pickup_community_area</td>
<td>fare</td>
<td>trip_start_month</td>
</tr>
<tr>
<td>trip_start_hour</td>
<td>trip_start_day</td>
<td>trip_start_timestamp</td>
</tr>
<tr>
<td>pickup_latitude</td>
<td>pickup_longitude</td>
<td>dropoff_latitude</td>
</tr>
<tr>
<td>dropoff_longitude</td>
<td>trip_miles</td>
<td>pickup_census_tract</td>
</tr>
<tr>
<td>dropoff_census_tract</td>
<td>payment_type</td>
<td>company</td>
</tr>
<tr>
<td>trip_seconds</td>
<td>dropoff_community_area</td>
<td>tips</td>
</tr>
</table>

## Jupyter Extensions をインストールする

注意：ローカルの Jupyter ノートブックで実行している場合は、Jupyter を実行する前に、これらの Jupyter 拡張機能を環境にインストールする必要があります。

```bash
jupyter nbextension enable --py widgetsnbextension --sys-prefix
jupyter nbextension install --py --symlink tensorflow_model_analysis --sys-prefix
jupyter nbextension enable --py tensorflow_model_analysis --sys-prefix
```

## TensorFlow Model Analysis (TFMA) をインストールする

これにより、すべての依存関係が取り込まれます。これには 1 分かかります。


In [ ]:
# Upgrade pip to the latest, and install TFMA.
!pip install -U pip
!pip install tensorflow-model-analysis

**ここで、以下のセルを実行する前にランタイムを再起動する必要があります。**

In [ ]:
# This setup was tested with TF 2.5 and TFMA 0.31 (using colab), but it should
# also work with the latest release.
import sys

# Confirm that we're using Python 3
assert sys.version_info.major==3, 'This notebook must be run using Python 3.'

import tensorflow as tf
print('TF version: {}'.format(tf.__version__))
import apache_beam as beam
print('Beam version: {}'.format(beam.__version__))
import tensorflow_model_analysis as tfma
print('TFMA version: {}'.format(tfma.__version__))

**注意：先に進む前に、上記の出力にエラーがないことを確認してください。エラーが表示される場合は、インストールを再実行してください。 また、次の手順に進む前に、必ずランタイム/カーネルを再起動してください。**

## ファイルを読み込む

必要なものがすべて含まれている tar ファイルをダウンロードします。 これには以下が含まれます。

- トレーニングと評価のデータセット
- データスキーマ
- 保存されたトレーニングモデルとサービングモデル（kerasと推定量）、および、保存された評価モデル（推定量）

In [ ]:
# Download the tar file from GCP and extract it
import io, os, tempfile
TAR_NAME = 'saved_models-2.2'
BASE_DIR = tempfile.mkdtemp()
DATA_DIR = os.path.join(BASE_DIR, TAR_NAME, 'data')
MODELS_DIR = os.path.join(BASE_DIR, TAR_NAME, 'models')
SCHEMA = os.path.join(BASE_DIR, TAR_NAME, 'schema.pbtxt')
OUTPUT_DIR = os.path.join(BASE_DIR, 'output')

!curl -O https://storage.googleapis.com/artifacts.tfx-oss-public.appspot.com/datasets/{TAR_NAME}.tar
!tar xf {TAR_NAME}.tar
!mv {TAR_NAME} {BASE_DIR}
!rm {TAR_NAME}.tar

print("Here's what we downloaded:")
!ls -R {BASE_DIR}

## スキーマを解析する

ダウンロードしたものの中には、[TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/)により作成されたデータのスキーマがあります。 TFMAで使用できるように、これを解析します。

In [ ]:
import tensorflow as tf
from google.protobuf import text_format
from tensorflow.python.lib.io import file_io
from tensorflow_metadata.proto.v0 import schema_pb2
from tensorflow.core.example import example_pb2

schema = schema_pb2.Schema()
contents = file_io.read_file_to_string(SCHEMA)
schema = text_format.Parse(contents, schema)

## スキーマを使用して TFRecord を作成する

TFMAにデータセットへのアクセスを許可ためにTFRecordsファイルを作成します。それぞれの特徴に正しい型を提供するために、スキーマを使用します。

In [ ]:
import csv

datafile = os.path.join(DATA_DIR, 'eval', 'data.csv')
reader = csv.DictReader(open(datafile, 'r'))
examples = []
for line in reader:
  example = example_pb2.Example()
  for feature in schema.feature:
    key = feature.name
    if feature.type == schema_pb2.FLOAT:
      example.features.feature[key].float_list.value[:] = (
          [float(line[key])] if len(line[key]) > 0 else [])
    elif feature.type == schema_pb2.INT:
      example.features.feature[key].int64_list.value[:] = (
          [int(line[key])] if len(line[key]) > 0 else [])
    elif feature.type == schema_pb2.BYTES:
      example.features.feature[key].bytes_list.value[:] = (
          [line[key].encode('utf8')] if len(line[key]) > 0 else [])
  # Add a new column 'big_tipper' that indicates if tips was > 20% of the fare. 
  # TODO(b/157064428): Remove after label transformation is supported for Keras.
  big_tipper = float(line['tips']) > float(line['fare']) * 0.2
  example.features.feature['big_tipper'].float_list.value[:] = [big_tipper]
  examples.append(example)

tfrecord_file = os.path.join(BASE_DIR, 'train_data.rio')
with tf.io.TFRecordWriter(tfrecord_file) as writer:
  for example in examples:
    writer.write(example.SerializeToString())

!ls {tfrecord_file}

## TFMA のセットアップと実行

TFMA は、TF keras モデル、一般的な TF2 シグネチャ API に基づくモデル、TF 推定器ベースのモデルなど、さまざまなモデル型をサポートしています。[get_started](https://www.tensorflow.org/tfx/model_analysis/get_started) ガイドには、サポートされているモデルタイプと制限事項の完全なリストがあります。この例では、keras ベースのモデルと、[`EvalSavedModel`](https://www.tensorflow.org/tfx/model_analysis/eval_saved_model)として保存された推定器ベースのモデルを構成する方法を示します。他の構成の例については、[FAQ](https://www.tensorflow.org/tfx/model_analysis/faq) を参照してください。

TFMA は、トレーニング時に使用されたメトリック（組み込みメトリック）と、モデルが TFMA 構成設定の一部として保存された後に定義されたメトリックの計算をサポートします。keras [セットアップ{/ a0}については、構成の一部としてメトリックとプロットを手動で追加する方法を示します（サポートされているメトリックとプロットについては、](https://www.tensorflow.org/tfx/model_analysis/setup)[メトリック](https://www.tensorflow.org/tfx/model_analysis/metrics)ガイドを参照してください）。推定器のセットアップには、モデルとともに保存された組み込みのメトリックを使用します。私たちのセットアップには、次のセクションでより詳細に説明するいくつかのスライス仕様も含まれています。

[`tfma.EvalConfig`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalConfig) と [`tfma.EvalSharedModel`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalSharedModel)を作成した後、`tfma.run_model_analysis`を使用して TFMA を実行できます。これにより、[`tfma.EvalResult`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalResult) が作成され、後でメトリックとプロットをレンダリングするために使用できます。

### Keras

In [ ]:
import tensorflow_model_analysis as tfma

# Setup tfma.EvalConfig settings
keras_eval_config = text_format.Parse("""
  ## Model information
  model_specs {
    # For keras (and serving models) we need to add a `label_key`.
    label_key: "big_tipper"
  }

  ## Post training metric information. These will be merged with any built-in
  ## metrics from training.
  metrics_specs {
    metrics { class_name: "ExampleCount" }
    metrics { class_name: "BinaryAccuracy" }
    metrics { class_name: "BinaryCrossentropy" }
    metrics { class_name: "AUC" }
    metrics { class_name: "AUCPrecisionRecall" }
    metrics { class_name: "Precision" }
    metrics { class_name: "Recall" }
    metrics { class_name: "MeanLabel" }
    metrics { class_name: "MeanPrediction" }
    metrics { class_name: "Calibration" }
    metrics { class_name: "CalibrationPlot" }
    metrics { class_name: "ConfusionMatrixPlot" }
    # ... add additional metrics and plots ...
  }

  ## Slicing information
  slicing_specs {}  # overall slice
  slicing_specs {
    feature_keys: ["trip_start_hour"]
  }
  slicing_specs {
    feature_keys: ["trip_start_day"]
  }
  slicing_specs {
    feature_values: {
      key: "trip_start_month"
      value: "1"
    }
  }
  slicing_specs {
    feature_keys: ["trip_start_hour", "trip_start_day"]
  }
""", tfma.EvalConfig())

# Create a tfma.EvalSharedModel that points at our keras model.
keras_model_path = os.path.join(MODELS_DIR, 'keras', '2')
keras_eval_shared_model = tfma.default_eval_shared_model(
    eval_saved_model_path=keras_model_path,
    eval_config=keras_eval_config)

keras_output_path = os.path.join(OUTPUT_DIR, 'keras')

# Run TFMA
keras_eval_result = tfma.run_model_analysis(
    eval_shared_model=keras_eval_shared_model,
    eval_config=keras_eval_config,
    data_location=tfrecord_file,
    output_path=keras_output_path)

### Estimator

In [ ]:
import tensorflow_model_analysis as tfma

# Setup tfma.EvalConfig settings
estimator_eval_config = text_format.Parse("""
  ## Model information
  model_specs {
    # To use EvalSavedModel set `signature_name` to "eval".
    signature_name: "eval"
  }

  ## Post training metric information. These will be merged with any built-in
  ## metrics from training.
  metrics_specs {
    metrics { class_name: "ConfusionMatrixPlot" }
    # ... add additional metrics and plots ...
  }

  ## Slicing information
  slicing_specs {}  # overall slice
  slicing_specs {
    feature_keys: ["trip_start_hour"]
  }
  slicing_specs {
    feature_keys: ["trip_start_day"]
  }
  slicing_specs {
    feature_values: {
      key: "trip_start_month"
      value: "1"
    }
  }
  slicing_specs {
    feature_keys: ["trip_start_hour", "trip_start_day"]
  }
""", tfma.EvalConfig())

# Create a tfma.EvalSharedModel that points at our eval saved model.
estimator_base_model_path = os.path.join(
    MODELS_DIR, 'estimator', 'eval_model_dir')
estimator_model_path = os.path.join(
    estimator_base_model_path, os.listdir(estimator_base_model_path)[0])
estimator_eval_shared_model = tfma.default_eval_shared_model(
    eval_saved_model_path=estimator_model_path,
    eval_config=estimator_eval_config)

estimator_output_path = os.path.join(OUTPUT_DIR, 'estimator')

# Run TFMA
estimator_eval_result = tfma.run_model_analysis(
    eval_shared_model=estimator_eval_shared_model,
    eval_config=estimator_eval_config,
    data_location=tfrecord_file,
    output_path=estimator_output_path)

## メトリックとプロットの視覚化

評価を実行したので、次に TFMAを使用した視覚化を見てみましょう。次の例では、keras モデルで評価を実行した結果を視覚化します。推定器ベースのモデルを表示するには、`eval_result {/ code1}変数を指すように<code data-md-type="codespan">eval_result`を更新します。

In [ ]:
eval_result = keras_eval_result
# eval_result = estimator_eval_result

### メトリックのレンダリング

メトリックを表示するには、[`tfma.view.render_slicing_metrics`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/view/render_slicing_metrics)を使用します。

デフォルトでは、ビューには`Overall`スライスが表示されます。特定のスライスを表示するには、列の名前を使用するか（slicing_columnを設定）、 [`tfma.SlicingSpec`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/SlicingSpec)を指定します。

メトリックの視覚化は、次の相互作用をサポートします。

- パンを実行するにはクリックしてドラッグします
- スクロールしてズーム
- 右クリックしてビューをリセットします
- データポイントにカーソルを合わせると、詳細が表示されます
- 下部の選択を使用して、4つの異なるタイプのビューから選択します。

たとえば、前述の`slicing_specs`の特徴`trip_start_hour`を確認するように`slicing_column`を設定します。

In [ ]:
tfma.view.render_slicing_metrics(eval_result, slicing_column='trip_start_hour')

### スライスの概要

スライスの数が少ない場合、デフォルトの視覚化は**スライスの概要**です。 各スライスのメトリックの値が表示されます。 上記で`trip_start_hour`を選択したため、1時間ごとの精度や AUC などの指標が表示され、特定の時間に固有の問題を探し、他の時間に固有の問題を探すことができません。

上記の視覚化で：

- 列ヘッダーをクリックして、特徴 `trip_start_hours`である特徴列を並べ替えてみてください
- 精度で並べ替えてみてください。**例を使用した一部の時間の精度が0であることに注意してください。これは、問題を示している可能性があります**

このグラフでは、スライス内のさまざまなメトリックを選択して表示することもできます。

- [表示]メニューからさまざまなメトリックを選択してみてください
- [表示]メニューで[リコール]を選択してみてください。**例を使用した一部の時間のリコールが0であることに注意してください。これは、問題を示している可能性があります**

例の数が少ないスライス、つまり「重み」を除外するためのしきい値を設定することもできます。最小数の例を入力するか、スライダーを使用して設定します。

### メトリックヒストグラム

このビューは、代替の視覚化として**メトリックヒストグラム**もサポートします。このビューは、スライスの数が多い場合のデフォルトです。結果はバケットに分割され、スライス数/総重量/両方を視覚化できます。列ヘッダーをクリックすると、列を並べ替えることができます。重みが小さいスライスは、しきい値を設定することで除外できます。灰色のバンドをドラッグすると、さらにフィルタリングを適用できます。範囲をリセットするには、バンドをダブルクリックします。フィルタリングを使用して、ビジュアライゼーションテーブルとメトリックテーブルの外れ値を削除することもできます 線形スケールから対数スケールに切り替えるには歯車アイコンをクリックします。

- ビジュアライゼーションメニューで「メトリックヒストグラム」を選択してみてください


### その他のスライス

最初の`tfma.EvalConfig`は`slicing_specs`の全リストを作成しました。これは、`tfma.view.render_slicing_metrics`に渡されたスライス情報を更新することで視覚化できます。ここでは、`trip_start_day`スライス（曜日）を選択します。 <strong data-md-type="double_emphasis">{nbsp}`trip_start_day` を <code data-md-type="codespan">trip_start_month</code>に変更し、もう一度レンダリングしてさまざまなスライスを調べてみてください。</strong>

In [ ]:
tfma.view.render_slicing_metrics(eval_result, slicing_column='trip_start_day')

また、TFMA は、特徴の組み合わせを分析するための特徴クロスの作成もサポートしています。元の設定では、`trip_start_hour` と `trip_start_day`のクロスが作成されました。

In [ ]:
tfma.view.render_slicing_metrics(
    eval_result,
    slicing_spec=tfma.SlicingSpec(
        feature_keys=['trip_start_hour', 'trip_start_day']))

2つの列をクロスさせると、多くの組み合わせが作成されます。 クロスを絞り込んで、**正午から始まるトリップ**のみを確認します。 次に、ビジュアライゼーションから`binary_accuracy `を選択します。

In [ ]:
tfma.view.render_slicing_metrics(
    eval_result,
    slicing_spec=tfma.SlicingSpec(
        feature_keys=['trip_start_day'], feature_values={'trip_start_hour': '12'}))

### プロットのレンダリング

トレーニング後の`metric_specs`として`tfma.EvalConfig`に追加されたプロットは、[`tfma.view.render_plot`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/view/render_plot)を使用して表示できます。

メトリックと同様に、プロットはスライスごとに表示できます。メトリックとは異なり、特定のスライス値のプロットのみを表示できるため、`tfma.SlicingSpec`を使用し、スライス特徴の名前と値の両方を指定する必要があります。スライスが指定されていない場合は、`Overall` スライスのプロットが使用されます。

以下の例では、`trip_start_hour:1` スライスに対して計算された`CalibrationPlot` と `ConfusionMatrixPlot`プロットを表示しています。

In [ ]:
tfma.view.render_plot(
    eval_result,
    tfma.SlicingSpec(feature_values={'trip_start_hour': '1'}))

## モデルのパフォーマンスを経時的に追跡する

トレーニングデータセットはモデルのトレーニングに使用されます。これはテストデータセットを代表するデータとして実環境でモデルに送信されます。ただし、推論リクエストのデータはトレーニングデータと同じままである場合がありますが、多くの場合、モデルのパフォーマンスが変化するほど変化することがあります。

つまり、モデルのパフォーマンスを継続的に監視および測定して、変更を認識して対応する必要があります。TFMA がどのように役立つかを見てみましょう。

3つの異なるモデルの実行を読み込み、TFMA を使用して、[`render_time_series`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/view/render_time_series)を使用して比較します。

In [ ]:
# Note this re-uses the EvalConfig from the keras setup.

# Run eval on each saved model
output_paths = []
for i in range(3):
  # Create a tfma.EvalSharedModel that points at our saved model.
  eval_shared_model = tfma.default_eval_shared_model(
      eval_saved_model_path=os.path.join(MODELS_DIR, 'keras', str(i)),
      eval_config=keras_eval_config)

  output_path = os.path.join(OUTPUT_DIR, 'time_series', str(i))
  output_paths.append(output_path)

  # Run TFMA
  tfma.run_model_analysis(eval_shared_model=eval_shared_model,
                          eval_config=keras_eval_config,
                          data_location=tfrecord_file,
                          output_path=output_path)

まず、昨日モデルをトレーニングしてデプロイしたとし、今日入ってくる新しいデータでモデルがどのように機能するかを確認します。AUC を表示して視覚化します。 UI から次のことができます。

- [メトリックシリーズの追加]メニューを使用して、他のメトリックを追加します。
- x をクリックして不要なグラフを閉じます。
- 詳細を表示するには、データポイント（グラフの線分の端）にカーソルを合わせます。

注意：メトリック系列グラフでは、X 軸は調査しているモデル実行のモデルディレクトリ名です。これらの名前自体には意味がありません。

In [ ]:
eval_results_from_disk = tfma.load_eval_results(output_paths[:2])

tfma.view.render_time_series(eval_results_from_disk)

一日経過したとし、前の2日間と比較して、今日入ってくる新しいデータでモデルがどのように機能しているかを確認します。

In [ ]:
eval_results_from_disk = tfma.load_eval_results(output_paths)

tfma.view.render_time_series(eval_results_from_disk)

## モデルの検証

TFMA は、複数のモデルを同時に評価するように構成できます。通常、これは、新しいモデルをベースライン（現在提供されているモデルなど）と比較し、ベースラインと比較したメトリック（AUC など）のパフォーマンスの差異を判断するために行われます。[しきい値](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/MetricThreshold)が設定されている場合、TFMA はパフォーマンスが期待値と一致するかどうかを示す[`tfma.ValidationResult`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/ValidationResult)レコードを生成します。

候補とベースラインの2つのモデルを比較するために、keras 評価を再構成してみましょう。また、AUC メトリックに [`tmfa.MetricThreshold`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/MetricThreshold)を設定することにより、ベースラインに対する候補のパフォーマンスを検証します。

In [ ]:
# Setup tfma.EvalConfig setting
eval_config_with_thresholds = text_format.Parse("""
  ## Model information
  model_specs {
    name: "candidate"
    # For keras we need to add a `label_key`.
    label_key: "big_tipper"
  }
  model_specs {
    name: "baseline"
    # For keras we need to add a `label_key`.
    label_key: "big_tipper"
    is_baseline: true
  }

  ## Post training metric information
  metrics_specs {
    metrics { class_name: "ExampleCount" }
    metrics { class_name: "BinaryAccuracy" }
    metrics { class_name: "BinaryCrossentropy" }
    metrics {
      class_name: "AUC"
      threshold {
        # Ensure that AUC is always > 0.9
        value_threshold {
          lower_bound { value: 0.9 }
        }
        # Ensure that AUC does not drop by more than a small epsilon
        # e.g. (candidate - baseline) > -1e-10 or candidate > baseline - 1e-10
        change_threshold {
          direction: HIGHER_IS_BETTER
          absolute { value: -1e-10 }
        }
      }
    }
    metrics { class_name: "AUCPrecisionRecall" }
    metrics { class_name: "Precision" }
    metrics { class_name: "Recall" }
    metrics { class_name: "MeanLabel" }
    metrics { class_name: "MeanPrediction" }
    metrics { class_name: "Calibration" }
    metrics { class_name: "CalibrationPlot" }
    metrics { class_name: "ConfusionMatrixPlot" }
    # ... add additional metrics and plots ...
  }

  ## Slicing information
  slicing_specs {}  # overall slice
  slicing_specs {
    feature_keys: ["trip_start_hour"]
  }
  slicing_specs {
    feature_keys: ["trip_start_day"]
  }
  slicing_specs {
    feature_keys: ["trip_start_month"]
  }
  slicing_specs {
    feature_keys: ["trip_start_hour", "trip_start_day"]
  }
""", tfma.EvalConfig())

# Create tfma.EvalSharedModels that point at our keras models.
candidate_model_path = os.path.join(MODELS_DIR, 'keras', '2')
baseline_model_path = os.path.join(MODELS_DIR, 'keras', '1')
eval_shared_models = [
  tfma.default_eval_shared_model(
      model_name=tfma.CANDIDATE_KEY,
      eval_saved_model_path=candidate_model_path,
      eval_config=eval_config_with_thresholds),
  tfma.default_eval_shared_model(
      model_name=tfma.BASELINE_KEY,
      eval_saved_model_path=baseline_model_path,
      eval_config=eval_config_with_thresholds),
]

validation_output_path = os.path.join(OUTPUT_DIR, 'validation')

# Run TFMA
eval_result_with_validation = tfma.run_model_analysis(
    eval_shared_models,
    eval_config=eval_config_with_thresholds,
    data_location=tfrecord_file,
    output_path=validation_output_path)

ベースラインに対して1つ以上のモデルで評価を実行すると、TFMA は、評価中に計算されたすべてのメトリックの差分メトリックを自動的に追加します。これらのメトリックは、対応するメトリックにちなんで名付けられていますが、メトリック名に`_diff`が追加されます。

実行により生成されたメトリックを見てみましょう。

In [ ]:
tfma.view.render_time_series(eval_result_with_validation)

次に、検証チェックからの出力を見てみましょう。検証結果を表示するには、[`tfma.load_validator_result`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/load_validation_result)を使用します。 この例では、AUC がしきい値を下回っているため、検証は不合格になります。

In [ ]:
validation_result = tfma.load_validation_result(validation_output_path)
print(validation_result.validation_ok)

# Copyright © 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

注意：このWeb サイトは、シカゴ市の公式 Web サイト www.cityofchicago.org で公開されたデータを変更して使用するアプリケーションを提供します。シカゴ市は、この Web サイトで提供されるデータの内容、正確性、適時性、または完全性について一切の表明を行いません。この Web サイトで提供されるデータは、随時変更される可能性があります。かかる Web サイトで提供されるデータはユーザーの自己責任で利用されるものとします。